## Bayut Oman Scraping

In [6]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import time

In [7]:
# 1. Define the base URL of the website
base_url = "https://www.bayut.om/en/oman/properties-for-sale/"
# Initialize the current page URL to the base URL
current_page_url = "https://www.bayut.om/en/oman/properties-for-sale/"
#headers={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.google.com/',
    'DNT': '1'
}

In [3]:
response=requests.get(current_page_url)
response.status_code

200

In [8]:
#Create an empty list to store all listings
bayut_data = []

print(f"Starting to scrape from: {current_page_url}")

Starting to scrape from: https://www.bayut.om/en/oman/properties-for-sale/


In [9]:
print(response.status_code)
print(response.text[:1000])  # show first 1000 characters of the page

200
<!DOCTYPE html><html lang="en" dir="ltr" itemscope="" recommenderthreelistingsondetailpages="recommended_only" itemType="http://schema.org/WebPage"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=0"/><link rel="dns-prefetch" href="http://LL8IZ711CS-dsn.algolia.net"/><link rel="dns-prefetch" href="https://www.googletagmanager.com"/><link rel="dns-prefetch" href="https://www.google-analytics.com"/><link rel="dns-prefetch" href="https://images.bayut.com"/><link rel="apple-touch-icon" sizes="180x180" href="/assets/apple-touch-icon.7919a66ff6f58dbfb1d2e33cb19b5a80.png"/><link rel="icon" type="image/png" sizes="16x16" href="/assets/favicon-16x16.0e9b01e26e8e124b4b030829ed6e5bfb.png"/><link rel="icon" type="image/png" sizes="32x32" href="/assets/favicon-32x32.20ce04f8a637e94fd0dc5ca4eeab1dce.png"/><link rel="manifest" href="/assets/9747ba91762d335f81891519d885a64f.json" crossorigin="use-credentials"/><link rel="mask-icon" hr

In [13]:
# Loop through pages until no 'next' button is found
while True:
    print(f"Scraping page: {current_page_url}")
    try:
        response = requests.get(current_page_url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {current_page_url}: {e}")
        break # Exit loop on request error
    soup = BeautifulSoup(response.content, 'html.parser')
    listings = soup.find_all('div', class_ ='_475e888a')

    for listing in listings:
        # Extract basic listing info
        title_elem = listing.find('span', class_='_19e94678 e0abc2de')
        title = title_elem.text.strip() if title_elem else None
            
        price_elem = listing.find('span', class_='dc381b54')
        price = price_elem.text.strip() if price_elem else None
            
        location_elem = listing.find('h3', class_='_4402bd70')
        location = location_elem.text.strip() if location_elem else None

        listing_type = listing.find_all(".breadcrumbs li:nth-child(3)")
        listingtype = listing_type.text.strip() if listing_type else None
        
        number_room = listing.find('span', attrs={'aria-label': 'Beds'})
        room_num= number_room.text.strip() if number_room else None
            
        prop_size = listing.find('h4', class_="cfac7e1b _85ddb82f")
        size = prop_size.text if prop_size else None

          # Store in dictionary
        bayut_data.append({
                "source": "Bayut",
                "title": title,
                "price": price,
                "location": location,
                "room number" : room_num,
                "size": size,
                "listing type" : listingtype
        })
            
        next_page_anchor = soup.find('a', class_='_95dd93c1', attrs={'title': 'Next'}, href=True)
        if next_page_anchor:
            next_page_relative_url = next_page_anchor['href']
            new_url = requests.compat.urljoin(base_url, next_page_relative_url)

    # Prevent infinite loop if same page repeats
    if new_url == current_page_url:
        print("no next page")
        break
    else:
        current_page_url = new_url
        time.sleep(1)

Scraping page: https://www.bayut.om/en/oman/properties-for-sale/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-2/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-3/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-4/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-5/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-6/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-7/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-8/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-9/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-10/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-11/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-12/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-13/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/pag

In [14]:
print("Total listings collected:", len(bayut_data))

Total listings collected: 1809


In [15]:
df = pd.DataFrame(bayut_data)
df

,source,title,price,location,room number,size,listing type
0,Bayut,Villa,"1,200","Al Buraimi, Al Buraimi",5,"1,500 Sq. M.",None
1,Bayut,Apartment,"1,200","Mahut, Al Wusta",3,"1,000 Sq. M.",None
2,Bayut,Villa,"1,500","Qantab, Muscat",5,"1,500 Sq. M.",None
3,Bayut,Villa,"1,500","Al Sifah, Muscat",5,"1,000 Sq. M.",None
4,Bayut,Villa,"1,000","Hamra, Al Dakhiliya",4,"1,000 Sq. M.",None
...,...,...,...,...,...,...,...
1804,Bayut,Villa,"1,000","Haima, Al Wusta",4,"1,000 Sq. M.",None
1805,Bayut,Apartment,600,"Mahut, Al Wusta",1,10 Sq. M.,None
1806,Bayut,Residential Land,"5,800","Nizwa, Al Dakhiliya",None,600 Sq. M.,None
1807,Bayut,Villa,"491,410","Yiti, Muscat",4,384 Sq. M.,None


In [5]:
df = pd.DataFrame(all_apt)
df

,title,location,price,size,listing_type
0,5 Bedrooms Villa for Sale in Oman,"Al Buraimi, Al Buraimi","OMR1,200","1,500 Sq. M.",Villa
1,5 Bedrooms Villa For Sale in Omani City,"Qantab, Muscat","OMR1,500","1,500 Sq. M.",Villa
2,"3 Bedrooms Apartment For Sale Mahut, Al Wusta","Mahut, Al Wusta","OMR1,200","1,000 Sq. M.",Apartment
3,"5 Bedrooms Villa For Sale in Al Sifah, Muscat","Al Sifah, Muscat","OMR1,500","1,000 Sq. M.",Villa
4,"4 Bedrooms Villa For Sale in Hamra, Al Dakhiliya","Hamra, Al Dakhiliya","OMR1,000","1,000 Sq. M.",Villa
...,...,...,...,...,...
183,"3 Bedrooms Villa For Sale Yiti, Muscat","Yiti, Muscat","OMR238,891",253 Sq. M.,Villa
184,4 Bedroom Villa For Sale in Sultan Haitham Cit...,"Sultan Haitham City, Muscat","OMR340,000",388 Sq. M.,Villa
185,2 Bedroom Apartment For Sale in The Wave (Almo...,"The Wave (Almouj), Muscat","OMR160,000",200 Sq. M.,Apartment
186,"Residential Land For Sale in Ibra, Al Sharqiya","Ibra, Al Sharqiya","OMR9,500",680 Sq. M.,Residential Land


In [16]:
if bayut_data:  # only write if there's data
    df_bayut = pd.DataFrame(bayut_data)
    df_bayut.to_csv("bayut_detailed.csv", index=False)
else:
    print("No data collected. CSV will not be created.")

In [17]:
df.head()

,source,title,price,location,room number,size,listing type
0,Bayut,Villa,"1,200","Al Buraimi, Al Buraimi",5,"1,500 Sq. M.",None
1,Bayut,Apartment,"1,200","Mahut, Al Wusta",3,"1,000 Sq. M.",None
2,Bayut,Villa,"1,500","Qantab, Muscat",5,"1,500 Sq. M.",None
3,Bayut,Villa,"1,500","Al Sifah, Muscat",5,"1,000 Sq. M.",None
4,Bayut,Villa,"1,000","Hamra, Al Dakhiliya",4,"1,000 Sq. M.",None
